In [1]:
import pandas as pd

#### Functions

In [ ]:
def quality_of_service(vehicles, antennas, a):
    """
    Calculate the quality of service (QoS) based on the provided data.

    Parameters:
    - vehicles (DataFrame): A DataFrame containing vehicle-related data.
    - antennas (DataFrame): A DataFrame containing antenna-related data.
    - a (float): The threshold value for the QoS calculation.

    Returns:
    - QoS (float): The calculated quality of service as the proportion of vehicles
      with a QoS value greater than or equal to the given threshold 'a'.
    """

    # Merge vehicle and antenna data
    vehicles = vehicles.merge(antennas, how="left")
    vehicles["Connection"] = vehicles["Permanence (s)"] * vehicles["ON"]

    # Group and aggregate data by "ID"
    df_ = vehicles[["ID", "Permanence (s)", "Connection"]].groupby("ID").sum().reset_index()
    df_["Delta"] = df_["Connection"] / df_["Permanence (s)"]

    # Calculate the QoS based on the threshold 'a'
    m = df_["Delta"] >= a
    QoS = m.sum() / df_.shape[0]

    return QoS


In [ ]:
def verification(df, df_RSU, b, a):
    """
    Verify if the minimum quality of service (QoS) can be guaranteed based on given parameters.

    Parameters:
    - df (DataFrame): A DataFrame containing vehicle-related data.
    - df_RSU (DataFrame): A DataFrame containing antenna-related data.
    - b (float): The minimum required quality of service (QoS) threshold.
    - a (float): The threshold value for the QoS calculation.

    Returns:
    - result (bool): True if the minimum QoS can be guaranteed, False otherwise.
    
    This function calculates the QoS using the provided data and compares it to the
    minimum required QoS threshold 'b'. If the calculated QoS is greater than or
    equal to 'b', it returns True, indicating that the minimum QoS can be guaranteed.
    Otherwise, it returns False and prints a message indicating that no minimum QoS
    guarantee can be provided.
    """
    
    # Calculate the quality of service based on the provided data
    beta_ = quality_of_service(df.copy(), df_RSU.copy(), a)
    
    
    # Check if the calculated QoS is greater than or equal to the minimum required threshold 'b'
    if beta_ >= b:
        print(f"Minimum Quality Guaranteed: {df_RSU["ON"].sum()}")
        return True
    else:
        print(f"No minimum Quality Guarantee")
        return False


In [ ]:
def on_off_heuristic(df_RSU, df_, df, cut, col, beta, alfa):
    """
    Apply an on-off heuristic algorithm to optimize RSU deployment.

    Parameters:
    - df_RSU (DataFrame): A DataFrame containing RSU-related data.
    - df_ (DataFrame): A DataFrame with additional data used for optimization.
    - df (DataFrame): A DataFrame containing vehicle-related data.
    - cut (list): A list of cutoff values for optimization iterations.
    - col (str): The column in 'df_' to use for optimization.
    - beta (float): The minimum required quality of service (QoS) threshold.
    - alfa (float): The threshold value for the QoS calculation.

    Returns:
    - optimized_RSU (DataFrame): A DataFrame with optimized RSU deployment,
      indicating which RSUs are turned on (1) and off (0).

    This function applies an on-off heuristic algorithm to optimize the deployment
    of RSUs based on given data and parameters. It starts with RSUs turned on, and
    iteratively adjusts the deployment based on cutoff values, 'col' for optimization,
    and quality of service requirements. The function returns a DataFrame indicating
    the optimized RSU deployment with 'ON' values (1 for on and 0 for off) for each RSU.
    """
    
    # Merge additional data from 'df_' into the 'df_RSU' DataFrame
    df_RSU = df_RSU.merge(df_[['X', 'Y', col]], how="left")
    
    # Sort 'df_RSU' based on the specified 'col' for optimization
    df_RSU.sort_values(col, inplace=True)

    # Reset the index of 'df_RSU'
    df_RSU.reset_index(drop=True, inplace=True)

    # Perform an initial verification of the quality of service (QoS)
    g = verification(df, df_RSU, beta, alfa)

    # Iterate through cutoff values
    for x in cut:
        m = df_RSU[col] <= x
        df_RSU.loc[m, "ON"] = 0
        g = verification(df, df_RSU, beta, alfa)

    i = (df_RSU["ON"] == 0).sum() - 1
    
    # Continue optimizing RSU deployment while QoS requirements are met
    while g:
        i += 1
        df_RSU.loc[i, "ON"] = 0
        g = verification(df, df_RSU, beta, alfa)

    # Set the last RSU to be turned on
    df_RSU.loc[i, "ON"] = 1
    g = verification(df, df_RSU, beta, alfa)

    # Return the optimized RSU deployment as a DataFrame
    return df_RSU[['X', 'Y', "ON"]]


#### On-Off Heuristic

In [ ]:
# Set the values of 'beta' and 'alfa'
beta = 0.3
alfa = 0.5

# Read a Parquet file named 'vehicles.parquet' and store it in the 'df' DataFrame
df = pd.read_parquet("files\\vehicles.parquet")

# Read a Parquet file named 'antennas.parquet' and store it in the 'df_RSU' DataFrame
df_RSU = pd.read_parquet("files\\antennas.parquet")

# Set all values in the 'ON' column of the 'df_RSU' DataFrame to 1
df_RSU["ON"] = 1

# Create a new DataFrame 'df_' by grouping and aggregating data from 'df' based on 'X' and 'Y'
df_ = df[['X', 'Y', "ID", "Permanence (s)"]].groupby(['X', 'Y']).agg({"ID" : "count", "Permanence (s)" : "sum"}).reset_index()

# Calculate the mean 'Permanence' and add it as a new 'Mean' column in the 'df_' DataFrame
df_["Mean"] = df_["Permanence (s)"] / df_["ID"]

# Rename columns in the 'df_' DataFrame for clarity
df_.rename(columns = {"ID" : "Vehicles", "Permanence (s)" : "Permanence"}, inplace = True)

In [ ]:
# Define a list of cutoff values 'cut'
cut = [10000, 25000, 45000, 48000]

# Run the 'on_off_heuristic' function with specific inputs, using the 'Permanence' criterion
# to calculate a solution and store it in the 'sol' variable
sol = on_off_heuristic(df_RSU.copy(), df_.copy(), df.copy(), cut, "Permanence", beta, alfa)

# Save the solution 'sol' to a Parquet file named 'Solution_Permanence.parquet'
sol.to_parquet("files\\Solution_Permanence.parquet")

In [ ]:
# Define a different list of cutoff values 'cut'
cut = [10, 15, 20, 25, 26]

# Run the 'on_off_heuristic' function with specific inputs, using the 'Mean' criterion
# to calculate a solution and store it in the 'sol' variable
sol = on_off_heuristic(df_RSU.copy(), df_.copy(), df.copy(), cut, "Mean", beta, alfa)

# Save the solution 'sol' to a Parquet file named 'Solution_Mean.parquet'
sol.to_parquet("files\\Solution_Mean.parquet")


In [ ]:
# Define a different list of cutoff values 'cut'
cut = [100, 200, 500, 1000, 1500, 1750, 1900, 1950]

# Run the 'on_off_heuristic' function with specific inputs, using the 'Vehicles' criterion
# to calculate a solution and store it in the 'sol' variable
sol = on_off_heuristic(df_RSU.copy(), df_.copy(), df.copy(), cut, "Vehicles", beta, alfa)

# Save the solution 'sol' to a Parquet file named 'Solution_Vehicles.parquet'
sol.to_parquet("files\\Solution_Vehicles.parquet")